Here we are giving a brief introduction in working with IAMC-styled data with pandas and pandas-indexing.

In [1]:
import pandas as pd

# Test data set

For experimenting and easy testing `pandas-indexing` brings along the power sector generation and capacity of the HighRE illustrative modelling pathway from the IPCC AR6 scenario database in IAMC format.

In [2]:
from pandas_indexing.datasets import remindhighre_power


df = remindhighre_power()
df.head()

2005  \
model                 scenario                     region variable                        unit              
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_HighRE_Budg900 World  Capacity|Electricity|Biomass    GW      42.9843   
                                                          Capacity|Electricity|Coal       GW    1238.8766   
                                                          Capacity|Electricity|Gas        GW    1096.6736   
                                                          Capacity|Electricity|Geothermal GW       8.1129   
                                                          Capacity|Electricity|Hydro      GW     806.5684   

                                                                                                     2010  \
model                 scenario                     region variable                        unit              
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_HighRE_Budg900 World  Capacity|Electricity|Biomass    GW      62.4717   
                                                          Capacity|Electricity|Coal       GW    1525.9497   
                                                          Capacity|Electricity|Gas        GW    1343.1077   
                                                          Capacity|Electricity|Geothermal GW      12.9985   
                                                          Capacity|Electricity|Hydro      GW     963.3523   

                                                                                                     2015  \
model                 scenario                     region variable                        unit              
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_HighRE_Budg900 World  Capacity|Electricity|Biomass    GW      85.6698   
                                                          Capacity|Electricity|Coal       GW    1876.5954   
                                                          Capacity|Electricity|Gas        GW    1603.1686   
                                                          Capacity|Electricity|Geothermal GW      24.1612   
                                                          Capacity|Electricity|Hydro      GW    1239.6174   

                                                                                                     2020  \
model                 scenario                     region variable                        unit              
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_HighRE_Budg900 World  Capacity|Electricity|Biomass    GW     108.8834   
                                                          Capacity|Electricity|Coal       GW    2075.2321   
                                                          Capacity|Electricity|Gas        GW    1852.6485   
                                                          Capacity|Electricity|Geothermal GW      46.6573   
                                                          Capacity|Electricity|Hydro      GW    1500.4508   

                                                                                                     2025  \
model                 scenario                     region variable                        unit              
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_HighRE_Budg900 World  Capacity|Electricity|Biomass    GW     128.5653   
                                                          Capacity|Electricity|Coal       GW    1225.4509   
                                                          Capacity|Electricity|Gas        GW    1761.8057   
                                                          Capacity|Electricity|Geothermal GW      70.6469   
                                                          Capacity|Electricity|Hydro      GW    1669.3565   

                                                                                                     2030  \
model                 scenario                     region variable                        unit              
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_HighRE_Budg900 World  Capac

# Usage styles

`pandas-indexing` defines two different usage styles:

1. functions that can be imported from the toplevel module, like
   
   ```python
   from pandas_indexing import assignlevel
   assignlevel(df, unit="Mt CO2e/yr")
   ```
2. convenience accessors that are hooking into pandas as extensions, like
   
   ```python
   df.pix.assign(unit="Mt CO2e/yr)
   ```

Most of the functionality is available with both styles under slightly different names. I'll present the functional style here first (and add the alternative as comments)

In [3]:
from pandas_indexing.core import describelevel


describelevel(df)  # or: df.pix

Index:
 * model    : REMIND-MAgPIE 2.1-4.3 (1)
 * scenario : DeepElec_SSP2_HighRE_Budg900 (1)
 * region   : World (1)
 * variable : Capacity|Electricity|Biomass, ... (20)
 * unit     : GW, GWh/yr (2)

Columns:
 * <unnamed> : 2005, 2010, 2015, 2020, 2025, 2030, 2035, 2040, ... 2100 (16)


In [4]:
df.pix

Index:
 * model    : REMIND-MAgPIE 2.1-4.3 (1)
 * scenario : DeepElec_SSP2_HighRE_Budg900 (1)
 * region   : World (1)
 * variable : Capacity|Electricity|Biomass, ... (20)
 * unit     : GW, GWh/yr (2)

Columns:
 * <unnamed> : 2005, 2010, 2015, 2020, 2025, 2030, 2035, 2040, ... 2100 (16)

As one can see the IAMC format is defined by five index levels: `model`, `scenario`, `variable`, `unit` and `region`. In this data subset, we have a single `model`, `scenario` combination for one `region` and with several capacity `variable`s starting with `Capacity|Electricity|` and generation variables starting with `Secondary Energy|Electricity|`.

The data comes with two different units: `GW` and `GWh/yr`, (hopefully) for capacity and generation, respectively.

# Selecting data

For using pandas indexes effectively for computations, it makes sense to split the hierarchically variable index out into separate python variables: `generation` and `capacity`. The standard pandas tools for this job are `pd.DataFrame.loc` in conjunction with `pd.IndexSlice` or `pd.DataFrame.query`. 

`pandas_indexing` brings `ismatch` and `isin` to make this job as easy as possible.

In [5]:
from pandas_indexing import isin, ismatch  # no .idx equivalents

In [6]:
df.loc[ismatch(variable="Capacity|**"), 2030]

model                  scenario                      region  variable                         unit
REMIND-MAgPIE 2.1-4.3  DeepElec_SSP2_HighRE_Budg900  World   Capacity|Electricity|Biomass     GW        137.8452
                                                             Capacity|Electricity|Coal        GW        182.0149
                                                             Capacity|Electricity|Gas         GW       1584.4033
                                                             Capacity|Electricity|Geothermal  GW         83.4506
                                                             Capacity|Electricity|Hydro       GW       1786.0468
                                                             Capacity|Electricity|Nuclear     GW        275.5920
                                                             Capacity|Electricity|Oil         GW         90.2329
                                                             Capacity|Electricity|Other       GW        469.52

`ismatch` allows using a glob-like pattern to subset into one or multiple named levels, together with the standard `rename` method we can get cleaned up capacity and generation data easily:

In [7]:
generation = df.loc[ismatch(variable="Secondary Energy|**")].rename(
    index=lambda s: s.removeprefix("Secondary Energy|Electricity|")
)
generation

2005  \
model                 scenario                     region variable   unit                   
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_HighRE_Budg900 World  Biomass    GWh/yr  2.352778e+05   
                                                          Coal       GWh/yr  7.419778e+06   
                                                          Gas        GWh/yr  3.816028e+06   
                                                          Geothermal GWh/yr  6.041667e+04   
                                                          Hydro      GWh/yr  2.976944e+06   
                                                          Nuclear    GWh/yr  2.861861e+06   
                                                          Oil        GWh/yr  1.088861e+06   
                                                          Other      GWh/yr  0.000000e+00   
                                                          Solar      GWh/yr  4.083333e+03   
                                                          Wind       GWh/yr  1.091389e+05   

                                                                                     2010  \
model                 scenario                     region variable   unit                   
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_HighRE_Budg900 World  Biomass    GWh/yr  3.407222e+05   
                                                          Coal       GWh/yr  8.252667e+06   
                                                          Gas        GWh/yr  5.050500e+06   
                                                          Geothermal GWh/yr  9.677778e+04   
                                                          Hydro      GWh/yr  3.557028e+06   
                                                          Nuclear    GWh/yr  2.642750e+06   
                                                          Oil        GWh/yr  1.016972e+06   
                                                          Other      GWh/yr  0.000000e+00   
                                                          Solar      GWh/yr  8.325000e+04   
                                                          Wind       GWh/yr  5.946944e+05   

                                                                                     2015  \
model                 scenario                     region variable   unit                   
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_HighRE_Budg900 World  Biomass    GWh/yr  4.669444e+05   
                                                          Coal       GWh/yr  9.246333e+06   
                                                          Gas        GWh/yr  5.968778e+06   
                                                          Geothermal GWh/yr  1.799167e+05   
                                                          Hydro      GWh/yr  4.386694e+06   
                                                          Nuclear    GWh/yr  2.448556e+06   
                                                          Oil        GWh/yr  8.353889e+05   
                                                          Other      GWh/yr  4.444444e+02   
                                                          Solar      GWh/yr  2.616111e+05   
                                                          Wind       GWh/yr  8.323889e+05   

                                                                                     2020  \
model                 scenario                     region variable   unit                   
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_HighRE_Budg900 World  Biomass    GWh/yr  5.942778e+05   
                                                          Coal       GWh/yr  1.075747e+07   
                                                          Gas        GWh/yr  7.060083e+06   
                                                          Geothermal GWh/yr  3.474167e+05   
                                                          Hydro      GWh/yr  5.302056e+06   
                                                          Nuclear    GWh/yr  2.2141

Since this extraction of data is relatively common, `extractlevel` simplifies this by matching against a format-like template string:

In [8]:
from pandas_indexing import extractlevel, formatlevel


generation = extractlevel(df, variable="Secondary Energy|{carrier}|{fuel}", drop=True)
capacity = extractlevel(df, variable="Capacity|{carrier}|{fuel}", drop=True)
# or: df.pix.extract(variable="Secondary Energy|{carrier}|{fuel}")
generation

2005  \
model                 scenario                     region unit   carrier     fuel                       
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_HighRE_Budg900 World  GWh/yr Electricity Biomass     2.352778e+05   
                                                                             Coal        7.419778e+06   
                                                                             Gas         3.816028e+06   
                                                                             Geothermal  6.041667e+04   
                                                                             Hydro       2.976944e+06   
                                                                             Nuclear     2.861861e+06   
                                                                             Oil         1.088861e+06   
                                                                             Other       0.000000e+00   
                                                                             Solar       4.083333e+03   
                                                                             Wind        1.091389e+05   

                                                                                                 2010  \
model                 scenario                     region unit   carrier     fuel                       
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_HighRE_Budg900 World  GWh/yr Electricity Biomass     3.407222e+05   
                                                                             Coal        8.252667e+06   
                                                                             Gas         5.050500e+06   
                                                                             Geothermal  9.677778e+04   
                                                                             Hydro       3.557028e+06   
                                                                             Nuclear     2.642750e+06   
                                                                             Oil         1.016972e+06   
                                                                             Other       0.000000e+00   
                                                                             Solar       8.325000e+04   
                                                                             Wind        5.946944e+05   

                                                                                                 2015  \
model                 scenario                     region unit   carrier     fuel                       
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_HighRE_Budg900 World  GWh/yr Electricity Biomass     4.669444e+05   
                                                                             Coal        9.246333e+06   
                                                                             Gas         5.968778e+06   
                                                                             Geothermal  1.799167e+05   
                                                                             Hydro       4.386694e+06   
                                                                             Nuclear     2.448556e+06   
                                                                             Oil         8.353889e+05   
                                                                             Other       4.444444e+02   
                                                                             Solar       2.616111e+05   
                                                                             Wind        8.323889e+05   

                                                                                                 2020  \
model                 scenario                     region unit   carrier     fuel                       
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_HighRE_Budg900 World  GWh/yr Electricity Biomass     5.942778e+05   

The inverse operation is to combine strings back together with `formatlevel`:

In [9]:
formatlevel(generation, variable="Secondary Energy|{carrier}|{fuel}", drop=True)
# or: df.pix.format(variable="Secondary Energy|{carrier}|{fuel}")

2005  \
model                 scenario                     region unit   variable                                                
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_HighRE_Budg900 World  GWh/yr Secondary Energy|Electricity|Biomass     2.352778e+05   
                                                                 Secondary Energy|Electricity|Coal        7.419778e+06   
                                                                 Secondary Energy|Electricity|Gas         3.816028e+06   
                                                                 Secondary Energy|Electricity|Geothermal  6.041667e+04   
                                                                 Secondary Energy|Electricity|Hydro       2.976944e+06   
                                                                 Secondary Energy|Electricity|Nuclear     2.861861e+06   
                                                                 Secondary Energy|Electricity|Oil         1.088861e+06   
                                                                 Secondary Energy|Electricity|Other       0.000000e+00   
                                                                 Secondary Energy|Electricity|Solar       4.083333e+03   
                                                                 Secondary Energy|Electricity|Wind        1.091389e+05   

                                                                                                                  2010  \
model                 scenario                     region unit   variable                                                
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_HighRE_Budg900 World  GWh/yr Secondary Energy|Electricity|Biomass     3.407222e+05   
                                                                 Secondary Energy|Electricity|Coal        8.252667e+06   
                                                                 Secondary Energy|Electricity|Gas         5.050500e+06   
                                                                 Secondary Energy|Electricity|Geothermal  9.677778e+04   
                                                                 Secondary Energy|Electricity|Hydro       3.557028e+06   
                                                                 Secondary Energy|Electricity|Nuclear     2.642750e+06   
                                                                 Secondary Energy|Electricity|Oil         1.016972e+06   
                                                                 Secondary Energy|Electricity|Other       0.000000e+00   
                                                                 Secondary Energy|Electricity|Solar       8.325000e+04   
                                                                 Secondary Energy|Electricity|Wind        5.946944e+05   

                                                                                                                  2015  \
model                 scenario                     region unit   variable                                                
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_HighRE_Budg900 World  GWh/yr Secondary Energy|Electricity|Biomass     4.669444e+05   
                                                                 Secondary Energy|Electricity|Coal        9.246333e+06   
                                                                 Secondary Energy|Electricity|Gas         5.968778e+06   
                                                                 Secondary Energy|Electricity|Geothermal  1.799167e+05   
                                                                 Secondary Energy|Electricity|Hydro       4.386694e+06   
                                                                 Secondary Energy|Electricity|Nuclear     2.448556e+06   
                                                                 Secondary Energy|Electricity|Oil         8.353889e+05   
                                                                 Secondary Energy|Elec

With `generation` and `capacity` conveniently split into separate variables, we can calculate capacity factors (ratios of generation and capacity) directly, as long as we take care of removing the conflicting `unit` level. Similarly to `ismatch`, `isin` can be provided as an argument to `.loc[]` to select on named index levels with the difference that only exact matches are considered.

In [10]:
capacity_factor = generation.droplevel("unit") / 8760 / capacity.droplevel("unit")
capacity_factor.loc[isin(fuel=["Solar", "Wind", "Hydro", "Geothermal"]), 2030:2051]

2030  \
model                 scenario                     region carrier     fuel                   
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_HighRE_Budg900 World  Electricity Geothermal  0.849984   
                                                                      Hydro       0.394597   
                                                                      Solar       0.161156   
                                                                      Wind        0.263820   

                                                                                      2035  \
model                 scenario                     region carrier     fuel                   
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_HighRE_Budg900 World  Electricity Geothermal  0.850005   
                                                                      Hydro       0.394474   
                                                                      Solar       0.154237   
                                                                      Wind        0.275781   

                                                                                      2040  \
model                 scenario                     region carrier     fuel                   
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_HighRE_Budg900 World  Electricity Geothermal  0.849989   
                                                                      Hydro       0.396240   
                                                                      Solar       0.151994   
                                                                      Wind        0.289083   

                                                                                      2045  \
model                 scenario                     region carrier     fuel                   
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_HighRE_Budg900 World  Electricity Geothermal  0.849986   
                                                                      Hydro       0.398217   
                                                                      Solar       0.150698   
                                                                      Wind        0.302018   

                                                                                      2050  
model                 scenario                     region carrier     fuel                  
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_HighRE_Budg900 World  Electricity Geothermal  0.850021  
                                                                      Hydro       0.399716  
                                                                      Solar       0.149422  
                                                                      Wind        0.298163

Instead of dropping the `unit` level, there is also a set of unit-aware calculation functions, so that this full capacity factor calculation can be performed in very few steps (the unit aware calculation realizes correctly that the capacity factor is unit-less):

In [11]:
generation = extractlevel(df, variable="Secondary Energy|{carrier}|{fuel}", drop=True)
capacity = extractlevel(df, variable="Capacity|{carrier}|{fuel}", drop=True)
generation.pix.unitdiv(capacity)

2005  \
model                 scenario                     region unit carrier     fuel                   
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_HighRE_Budg900 World       Electricity Biomass     0.624410   
                                                                           Coal        0.683221   
                                                                           Gas         0.396947   
                                                                           Geothermal  0.849531   
                                                                           Hydro       0.421045   
                                                                           Nuclear     0.799454   
                                                                           Oil         0.242962   
                                                                           Other            NaN   
                                                                           Solar       0.173656   
                                                                           Wind        0.385930   

                                                                                           2010  \
model                 scenario                     region unit carrier     fuel                   
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_HighRE_Budg900 World       Electricity Biomass     0.622179   
                                                                           Coal        0.616954   
                                                                           Gas         0.428965   
                                                                           Geothermal  0.849339   
                                                                           Hydro       0.421212   
                                                                           Nuclear     0.745660   
                                                                           Oil         0.250517   
                                                                           Other            NaN   
                                                                           Solar       0.191417   
                                                                           Wind        0.374443   

                                                                                           2015  \
model                 scenario                     region unit carrier     fuel                   
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_HighRE_Budg900 World       Electricity Biomass     0.621779   
                                                                           Coal        0.562079   
                                                                           Gas         0.424722   
                                                                           Geothermal  0.849477   
                                                                           Hydro       0.403690   
                                                                           Nuclear     0.736736   
                                                                           Oil         0.256895   
                                                                           Other       0.149296   
                                                                           Solar       0.135744   
                                                                           Wind        0.230212   

                                                                                           2020  \
model                 scenario                     region unit carrier     fuel                   
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_HighRE_Budg900 World       Electricity Biomass     0.622625   
                                                                           Coal        0.591347   
                                                                           Gas         0.434726   
                             

Under the hood `isin` and `ismatch` generate `Selector` objects. They can be composed into complex queries intuitively, which are kept as a hierarchical structure of objects.

In [12]:
query = isin(fuel=["Coal", "Gas", "Nuclear"], unit="GW") & ~ismatch(fuel="S*")
query

And(a=Isin(filters={'fuel': ['Coal', 'Gas', 'Nuclear'], 'unit': 'GW'}), b=Not(a=Ismatch(filters={'fuel': 'S*'}, regex=False)))

For evaluating such a query one needs to pass in a data object to produce a boolean mask. Since pandas `.loc` indexer does exactly that, these queries work as expected.


In [13]:
query(capacity)

model                  scenario                      region  unit  carrier      fuel      
REMIND-MAgPIE 2.1-4.3  DeepElec_SSP2_HighRE_Budg900  World   GW    Electricity  Biomass       False
                                                                                Coal           True
                                                                                Gas            True
                                                                                Geothermal    False
                                                                                Hydro         False
                                                                                Nuclear        True
                                                                                Oil           False
                                                                                Other         False
                                                                                Solar         False
         

````{warning}
It is currently impossible to use a pandas boolean series **in front of** a selector; ie.
```python
(capacity[2030] > 250) & isin(variable=["Coal", "Gas", "Nuclear"], unit="GW")
```
will **fail**, it needs to be
```python
isin(variable=["Coal", "Gas", "Nuclear"], unit="GW") & (capacity[2030] > 250)
```
````

In [14]:
high_capacity_fossil = capacity.loc[
    isin(fuel=["Coal", "Gas", "Nuclear"], unit="GW") & (capacity[2030] > 250),
    :2041,
]
high_capacity_fossil

2005  \
model                 scenario                     region unit carrier     fuel                 
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_HighRE_Budg900 World  GW   Electricity Gas      1096.6736   
                                                                           Nuclear   408.3700   

                                                                                         2010  \
model                 scenario                     region unit carrier     fuel                 
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_HighRE_Budg900 World  GW   Electricity Gas      1343.1077   
                                                                           Nuclear   404.3094   

                                                                                         2015  \
model                 scenario                     region unit carrier     fuel                 
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_HighRE_Budg900 World  GW   Electricity Gas      1603.1686   
                                                                           Nuclear   379.1375   

                                                                                         2020  \
model                 scenario                     region unit carrier     fuel                 
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_HighRE_Budg900 World  GW   Electricity Gas      1852.6485   
                                                                           Nuclear   345.2239   

                                                                                         2025  \
model                 scenario                     region unit carrier     fuel                 
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_HighRE_Budg900 World  GW   Electricity Gas      1761.8057   
                                                                           Nuclear   310.1729   

                                                                                         2030  \
model                 scenario                     region unit carrier     fuel                 
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_HighRE_Budg900 World  GW   Electricity Gas      1584.4033   
                                                                           Nuclear   275.5920   

                                                                                         2035  \
model                 scenario                     region unit carrier     fuel                 
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_HighRE_Budg900 World  GW   Electricity Gas      1423.5393   
                                                                           Nuclear   234.7028   

                                                                                         2040  
model                 scenario                     region unit carrier     fuel                
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_HighRE_Budg900 World  GW   Electricity Gas      1289.4777  
                                                                           Nuclear   214.4376

The simple fact that this is an operation on `[]`, means that we can also use it to modify values in-place:

In [15]:
high_capacity_fossil.loc[isin(fuel="Gas"), 2030:] = 1000.0
high_capacity_fossil

2005  \
model                 scenario                     region unit carrier     fuel                 
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_HighRE_Budg900 World  GW   Electricity Gas      1096.6736   
                                                                           Nuclear   408.3700   

                                                                                         2010  \
model                 scenario                     region unit carrier     fuel                 
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_HighRE_Budg900 World  GW   Electricity Gas      1343.1077   
                                                                           Nuclear   404.3094   

                                                                                         2015  \
model                 scenario                     region unit carrier     fuel                 
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_HighRE_Budg900 World  GW   Electricity Gas      1603.1686   
                                                                           Nuclear   379.1375   

                                                                                         2020  \
model                 scenario                     region unit carrier     fuel                 
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_HighRE_Budg900 World  GW   Electricity Gas      1852.6485   
                                                                           Nuclear   345.2239   

                                                                                         2025  \
model                 scenario                     region unit carrier     fuel                 
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_HighRE_Budg900 World  GW   Electricity Gas      1761.8057   
                                                                           Nuclear   310.1729   

                                                                                        2030  \
model                 scenario                     region unit carrier     fuel                
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_HighRE_Budg900 World  GW   Electricity Gas      1000.000   
                                                                           Nuclear   275.592   

                                                                                         2035  \
model                 scenario                     region unit carrier     fuel                 
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_HighRE_Budg900 World  GW   Electricity Gas      1000.0000   
                                                                           Nuclear   234.7028   

                                                                                         2040  
model                 scenario                     region unit carrier     fuel                
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_HighRE_Budg900 World  GW   Electricity Gas      1000.0000  
                                                                           Nuclear   214.4376

Most methods in `pandas_indexing` do not care whether they are run on an index, a series or a dataframe, but will transiently take care of handing them down to the appropriate level:

In [16]:
fossil_series = (
    capacity.loc[isin(fuel=["Coal", "Gas", "Nuclear"]), [2030, 2040, 2050, 2060]]
    .rename_axis(columns="year")
    .stack()
)
fossil_series

model                  scenario                      region  unit  carrier      fuel     year
REMIND-MAgPIE 2.1-4.3  DeepElec_SSP2_HighRE_Budg900  World   GW    Electricity  Coal     2030     182.0149
                                                                                         2040       1.0729
                                                                                         2050       0.5029
                                                                                         2060       0.1836
                                                                                Gas      2030    1584.4033
                                                                                         2040    1289.4777
                                                                                         2050     562.8482
                                                                                         2060     349.9091
                                                  

In [17]:
fossil_series.loc[isin(year=[2030, 2050])]

model                  scenario                      region  unit  carrier      fuel     year
REMIND-MAgPIE 2.1-4.3  DeepElec_SSP2_HighRE_Budg900  World   GW    Electricity  Coal     2030     182.0149
                                                                                         2050       0.5029
                                                                                Gas      2030    1584.4033
                                                                                         2050     562.8482
                                                                                Nuclear  2030     275.5920
                                                                                         2050     156.7766
dtype: float64

In [18]:
isin(fossil_series.index, fuel="Nuclear")

array([False, False, False, False, False, False, False, False,  True,
        True,  True,  True])

# Selecting based on a multi-index

If we need pairs of data like `Coal` in 2030 and `Gas` in 2040 and `Nuclear` in 2040 and 2050, based on a given multiindex, then this defines right-oriented `semijoin` like:

In [19]:
idx = pd.MultiIndex.from_tuples(
    [("Coal", 2030), ("Gas", 2035), ("Nuclear", 2040), ("Nuclear", 2050)],
    names=["fuel", "year"],
)
idx

MultiIndex([(   'Coal', 2030),
            (    'Gas', 2035),
            ('Nuclear', 2040),
            ('Nuclear', 2050)],
           names=['fuel', 'year'])

In [20]:
from pandas_indexing import semijoin


semijoin(
    fossil_series, idx, how="right"
)  # or: fossil_series.pix.semijoin(idx, how="right")

fuel     year  model                  scenario                      region  unit  carrier    
Coal     2030  REMIND-MAgPIE 2.1-4.3  DeepElec_SSP2_HighRE_Budg900  World   GW    Electricity    182.0149
Gas      2035  NaN                    NaN                           NaN     NaN   NaN                 NaN
Nuclear  2040  REMIND-MAgPIE 2.1-4.3  DeepElec_SSP2_HighRE_Budg900  World   GW    Electricity    214.4376
         2050  REMIND-MAgPIE 2.1-4.3  DeepElec_SSP2_HighRE_Budg900  World   GW    Electricity    156.7766
dtype: float64

Since the `("Gas", 2035)` is not part of the original `fossil_series` it shows up as `NaN`s here, an inner join will skip it silently:

In [21]:
semijoin(fossil_series, idx, how="inner")

fuel     year  model                  scenario                      region  unit  carrier    
Coal     2030  REMIND-MAgPIE 2.1-4.3  DeepElec_SSP2_HighRE_Budg900  World   GW    Electricity    182.0149
Nuclear  2040  REMIND-MAgPIE 2.1-4.3  DeepElec_SSP2_HighRE_Budg900  World   GW    Electricity    214.4376
         2050  REMIND-MAgPIE 2.1-4.3  DeepElec_SSP2_HighRE_Budg900  World   GW    Electricity    156.7766
dtype: float64

It is also possible to get the inverted result, with only the not matching rows, with an `antijoin`

In [22]:
from pandas_indexing import antijoin


antijoin(fossil_series, idx)
# or: fossil_series.pix.antijoin(idx)

model                  scenario                      region  unit  carrier      fuel     year
REMIND-MAgPIE 2.1-4.3  DeepElec_SSP2_HighRE_Budg900  World   GW    Electricity  Coal     2040       1.0729
                                                                                         2050       0.5029
                                                                                         2060       0.1836
                                                                                Gas      2030    1584.4033
                                                                                         2040    1289.4777
                                                                                         2050     562.8482
                                                                                         2060     349.9091
                                                                                Nuclear  2030     275.5920
                                                  

# Projecting levels

Often after selecting the right subsets, ie the interesting `model` or `scenario` it makes sense to consolidate the data to a given set of `levels`. That is what `projectlevel` is used for:

In [23]:
from pandas_indexing import projectlevel


simple_fossil_series = projectlevel(fossil_series, ["fuel", "year"])
# or: fossil_series.pix.project(["fuel", "year"])
simple_fossil_series

fuel     year
Coal     2030     182.0149
         2040       1.0729
         2050       0.5029
         2060       0.1836
Gas      2030    1584.4033
         2040    1289.4777
         2050     562.8482
         2060     349.9091
Nuclear  2030     275.5920
         2040     214.4376
         2050     156.7766
         2060      92.0667
dtype: float64

`projectlevel` reduces the levels attached to a multiindex to the ones explicitly named. It is basically the complement to `droplevel` which removes the listed names

In [24]:
projectlevel(fossil_series, ["model", "scenario"]) == fossil_series.droplevel(
    ["carrier", "fuel", "unit", "region", "year"]
)

model                  scenario                    
REMIND-MAgPIE 2.1-4.3  DeepElec_SSP2_HighRE_Budg900    True
                       DeepElec_SSP2_HighRE_Budg900    True
                       DeepElec_SSP2_HighRE_Budg900    True
                       DeepElec_SSP2_HighRE_Budg900    True
                       DeepElec_SSP2_HighRE_Budg900    True
                       DeepElec_SSP2_HighRE_Budg900    True
                       DeepElec_SSP2_HighRE_Budg900    True
                       DeepElec_SSP2_HighRE_Budg900    True
                       DeepElec_SSP2_HighRE_Budg900    True
                       DeepElec_SSP2_HighRE_Budg900    True
                       DeepElec_SSP2_HighRE_Budg900    True
                       DeepElec_SSP2_HighRE_Budg900    True
dtype: bool

# Assigning to levels

`assignlevel` allows to modify individual values with helpful keyword arguments,

In [25]:
from pandas_indexing import assignlevel


assignlevel(df, variable="Updated|" + projectlevel(df.index, "variable"), unit="bla")
# or: df.pix.assign(variable=df.index.pix.project("variable"), unit="bla")

2005  \
model                 scenario                     region variable                                        unit                 
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_HighRE_Budg900 World  Updated|Capacity|Electricity|Biomass            bla   4.298430e+01   
                                                          Updated|Capacity|Electricity|Coal               bla   1.238877e+03   
                                                          Updated|Capacity|Electricity|Gas                bla   1.096674e+03   
                                                          Updated|Capacity|Electricity|Geothermal         bla   8.112900e+00   
                                                          Updated|Capacity|Electricity|Hydro              bla   8.065684e+02   
                                                          Updated|Capacity|Electricity|Nuclear            bla   4.083700e+02   
                                                          Updated|Capacity|Electricity|Oil                bla   5.112499e+02   
                                                          Updated|Capacity|Electricity|Other              bla   0.000000e+00   
                                                          Updated|Capacity|Electricity|Solar              bla   2.682400e+00   
                                                          Updated|Capacity|Electricity|Wind               bla   3.226040e+01   
                                                          Updated|Secondary Energy|Electricity|Biomass    bla   2.352778e+05   
                                                          Updated|Secondary Energy|Electricity|Coal       bla   7.419778e+06   
                                                          Updated|Secondary Energy|Electricity|Gas        bla   3.816028e+06   
                                                          Updated|Secondary Energy|Electricity|Geothermal bla   6.041667e+04   
                                                          Updated|Secondary Energy|Electricity|Hydro      bla   2.976944e+06   
                                                          Updated|Secondary Energy|Electricity|Nuclear    bla   2.861861e+06   
                                                          Updated|Secondary Energy|Electricity|Oil        bla   1.088861e+06   
                                                          Updated|Secondary Energy|Electricity|Other      bla   0.000000e+00   
                                                          Updated|Secondary Energy|Electricity|Solar      bla   4.083333e+03   
                                                          Updated|Secondary Energy|Electricity|Wind       bla   1.091389e+05   

                                                                                                                        2010  \
model                 scenario                     region variable                                        unit                 
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_HighRE_Budg900 World  Updated|Capacity|Electricity|Biomass            bla   6.247170e+01   
                                                          Updated|Capacity|Electricity|Coal               bla   1.525950e+03   
                                                          Updated|Capacity|Electricity|Gas                bla   1.343108e+03   
                                                          Updated|Capacity|Electricity|Geothermal         bla   1.299850e+01   
                                                          Updated|Capacity|Electricity|Hydro              bla   9.633523e+02   
                                                          Updated|Capacity|Electricity|Nuclear            bla   4.043094e+02   
                                                          Updated|Capacity|Electricity|Oil                bla   4.630961e+02   
                                                          Updated|Capacity|Electricity|Other              bla   0.000000e+00   
                       

This particular case is even more clearly handled with `formatlevel`:

In [26]:
from pandas_indexing import formatlevel


formatlevel(df, variable="Updated|{variable}", unit="bla")
# or: df.pix.format(variable=...)

2005  \
model                 scenario                     region variable                                        unit                 
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_HighRE_Budg900 World  Updated|Capacity|Electricity|Biomass            bla   4.298430e+01   
                                                          Updated|Capacity|Electricity|Coal               bla   1.238877e+03   
                                                          Updated|Capacity|Electricity|Gas                bla   1.096674e+03   
                                                          Updated|Capacity|Electricity|Geothermal         bla   8.112900e+00   
                                                          Updated|Capacity|Electricity|Hydro              bla   8.065684e+02   
                                                          Updated|Capacity|Electricity|Nuclear            bla   4.083700e+02   
                                                          Updated|Capacity|Electricity|Oil                bla   5.112499e+02   
                                                          Updated|Capacity|Electricity|Other              bla   0.000000e+00   
                                                          Updated|Capacity|Electricity|Solar              bla   2.682400e+00   
                                                          Updated|Capacity|Electricity|Wind               bla   3.226040e+01   
                                                          Updated|Secondary Energy|Electricity|Biomass    bla   2.352778e+05   
                                                          Updated|Secondary Energy|Electricity|Coal       bla   7.419778e+06   
                                                          Updated|Secondary Energy|Electricity|Gas        bla   3.816028e+06   
                                                          Updated|Secondary Energy|Electricity|Geothermal bla   6.041667e+04   
                                                          Updated|Secondary Energy|Electricity|Hydro      bla   2.976944e+06   
                                                          Updated|Secondary Energy|Electricity|Nuclear    bla   2.861861e+06   
                                                          Updated|Secondary Energy|Electricity|Oil        bla   1.088861e+06   
                                                          Updated|Secondary Energy|Electricity|Other      bla   0.000000e+00   
                                                          Updated|Secondary Energy|Electricity|Solar      bla   4.083333e+03   
                                                          Updated|Secondary Energy|Electricity|Wind       bla   1.091389e+05   

                                                                                                                        2010  \
model                 scenario                     region variable                                        unit                 
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_HighRE_Budg900 World  Updated|Capacity|Electricity|Biomass            bla   6.247170e+01   
                                                          Updated|Capacity|Electricity|Coal               bla   1.525950e+03   
                                                          Updated|Capacity|Electricity|Gas                bla   1.343108e+03   
                                                          Updated|Capacity|Electricity|Geothermal         bla   1.299850e+01   
                                                          Updated|Capacity|Electricity|Hydro              bla   9.633523e+02   
                                                          Updated|Capacity|Electricity|Nuclear            bla   4.043094e+02   
                                                          Updated|Capacity|Electricity|Oil                bla   4.630961e+02   
                                                          Updated|Capacity|Electricity|Other              bla   0.000000e+00   
                       

Both functions avoid having to rely on `reset_index`, `set_index` pairs, which are painful for large data, since `set_index` is expensive!

In [27]:
df.reset_index().assign(variable="Capacity").set_index(df.index.names)

2005  \
model                 scenario                     region variable unit                   
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_HighRE_Budg900 World  Capacity GW      4.298430e+01   
                                                                   GW      1.238877e+03   
                                                                   GW      1.096674e+03   
                                                                   GW      8.112900e+00   
                                                                   GW      8.065684e+02   
                                                                   GW      4.083700e+02   
                                                                   GW      5.112499e+02   
                                                                   GW      0.000000e+00   
                                                                   GW      2.682400e+00   
                                                                   GW      3.226040e+01   
                                                                   GWh/yr  2.352778e+05   
                                                                   GWh/yr  7.419778e+06   
                                                                   GWh/yr  3.816028e+06   
                                                                   GWh/yr  6.041667e+04   
                                                                   GWh/yr  2.976944e+06   
                                                                   GWh/yr  2.861861e+06   
                                                                   GWh/yr  1.088861e+06   
                                                                   GWh/yr  0.000000e+00   
                                                                   GWh/yr  4.083333e+03   
                                                                   GWh/yr  1.091389e+05   

                                                                                   2010  \
model                 scenario                     region variable unit                   
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_HighRE_Budg900 World  Capacity GW      6.247170e+01   
                                                                   GW      1.525950e+03   
                                                                   GW      1.343108e+03   
                                                                   GW      1.299850e+01   
                                                                   GW      9.633523e+02   
                                                                   GW      4.043094e+02   
                                                                   GW      4.630961e+02   
                                                                   GW      0.000000e+00   
                                                                   GW      4.961380e+01   
                                                                   GW      1.811785e+02   
                                                                   GWh/yr  3.407222e+05   
                                                                   GWh/yr  8.252667e+06   
                                                                   GWh/yr  5.050500e+06   
                                                                   GWh/yr  9.677778e+04   
                                                                   GWh/yr  3.557028e+06   
                                                                   GWh/yr  2.642750e+06   
                                                                   GWh/yr  1.016972e+06   
                                                                   GWh/yr  0.000000e+00   
                                                                   GWh/yr  8.325000e+04   
                                                                   GWh/yr  5.946944e+05   

                                                                             

# Examining level values and level combinations

We already encountered the possibility to get an overview of the available levels and their values with describelevel:

In [28]:
describelevel(df)  # or: df.idx

Index:
 * model    : REMIND-MAgPIE 2.1-4.3 (1)
 * scenario : DeepElec_SSP2_HighRE_Budg900 (1)
 * region   : World (1)
 * variable : Capacity|Electricity|Biomass, ... (20)
 * unit     : GW, GWh/yr (2)

Columns:
 * <unnamed> : 2005, 2010, 2015, 2020, 2025, 2030, 2035, 2040, ... 2100 (16)


Often it is necessary to get programmatic access to the unique values of one or more levels:

In [29]:
from pandas_indexing import uniquelevel


uniquelevel(df, "variable")
# or: df.pix.unique("variable")
# or in vanilla pandas: df.index.unique("variable")

Index(['Capacity|Electricity|Biomass', 'Capacity|Electricity|Coal',
       'Capacity|Electricity|Gas', 'Capacity|Electricity|Geothermal',
       'Capacity|Electricity|Hydro', 'Capacity|Electricity|Nuclear',
       'Capacity|Electricity|Oil', 'Capacity|Electricity|Other',
       'Capacity|Electricity|Solar', 'Capacity|Electricity|Wind',
       'Secondary Energy|Electricity|Biomass',
       'Secondary Energy|Electricity|Coal', 'Secondary Energy|Electricity|Gas',
       'Secondary Energy|Electricity|Geothermal',
       'Secondary Energy|Electricity|Hydro',
       'Secondary Energy|Electricity|Nuclear',
       'Secondary Energy|Electricity|Oil',
       'Secondary Energy|Electricity|Other',
       'Secondary Energy|Electricity|Solar',
       'Secondary Energy|Electricity|Wind'],
      dtype='object', name='variable')

In [30]:
uniquelevel(df, ["variable", "unit"])

MultiIndex([(           'Capacity|Electricity|Biomass',     'GW'),
            (              'Capacity|Electricity|Coal',     'GW'),
            (               'Capacity|Electricity|Gas',     'GW'),
            (        'Capacity|Electricity|Geothermal',     'GW'),
            (             'Capacity|Electricity|Hydro',     'GW'),
            (           'Capacity|Electricity|Nuclear',     'GW'),
            (               'Capacity|Electricity|Oil',     'GW'),
            (             'Capacity|Electricity|Other',     'GW'),
            (             'Capacity|Electricity|Solar',     'GW'),
            (              'Capacity|Electricity|Wind',     'GW'),
            (   'Secondary Energy|Electricity|Biomass', 'GWh/yr'),
            (      'Secondary Energy|Electricity|Coal', 'GWh/yr'),
            (       'Secondary Energy|Electricity|Gas', 'GWh/yr'),
            ('Secondary Energy|Electricity|Geothermal', 'GWh/yr'),
            (     'Secondary Energy|Electricity|Hydro', 'GWh/y

# BEWARE: Pitfalls

`concat` ignores level order, so make sure to `reorder_levels` them

In [31]:
pd.concat([simple_fossil_series, simple_fossil_series.swaplevel()])

Coal     2030        182.0149
         2040          1.0729
         2050          0.5029
         2060          0.1836
Gas      2030       1584.4033
         2040       1289.4777
         2050        562.8482
         2060        349.9091
Nuclear  2030        275.5920
         2040        214.4376
         2050        156.7766
         2060         92.0667
2030     Coal        182.0149
2040     Coal          1.0729
2050     Coal          0.5029
2060     Coal          0.1836
2030     Gas        1584.4033
2040     Gas        1289.4777
2050     Gas         562.8482
2060     Gas         349.9091
2030     Nuclear     275.5920
2040     Nuclear     214.4376
2050     Nuclear     156.7766
2060     Nuclear      92.0667
dtype: float64

In [32]:
pd.concat(
    [
        simple_fossil_series,
        simple_fossil_series.swaplevel().reorder_levels(
            simple_fossil_series.index.names
        ),
    ]
)

fuel     year
Coal     2030     182.0149
         2040       1.0729
         2050       0.5029
         2060       0.1836
Gas      2030    1584.4033
         2040    1289.4777
         2050     562.8482
         2060     349.9091
Nuclear  2030     275.5920
         2040     214.4376
         2050     156.7766
         2060      92.0667
Coal     2030     182.0149
         2040       1.0729
         2050       0.5029
         2060       0.1836
Gas      2030    1584.4033
         2040    1289.4777
         2050     562.8482
         2060     349.9091
Nuclear  2030     275.5920
         2040     214.4376
         2050     156.7766
         2060      92.0667
dtype: float64

Therefore, `pandas-indexing` brings a variant which does this automatically:

In [33]:
from pandas_indexing import concat


concat([simple_fossil_series, simple_fossil_series.swaplevel()])

fuel     year
Coal     2030     182.0149
         2040       1.0729
         2050       0.5029
         2060       0.1836
Gas      2030    1584.4033
         2040    1289.4777
         2050     562.8482
         2060     349.9091
Nuclear  2030     275.5920
         2040     214.4376
         2050     156.7766
         2060      92.0667
Coal     2030     182.0149
         2040       1.0729
         2050       0.5029
         2060       0.1836
Gas      2030    1584.4033
         2040    1289.4777
         2050     562.8482
         2060     349.9091
Nuclear  2030     275.5920
         2040     214.4376
         2050     156.7766
         2060      92.0667
dtype: float64

# Additional helpful multi-index helpers

MultiIndex rendering is often annoying to read, since the important information might get abbreviated away, then converting it into a dataframe is helpful

In [34]:
projectlevel(fossil_series.index, ["model", "scenario", "fuel"])

MultiIndex([('REMIND-MAgPIE 2.1-4.3', 'DeepElec_SSP2_HighRE_Budg900', ...),
            ('REMIND-MAgPIE 2.1-4.3', 'DeepElec_SSP2_HighRE_Budg900', ...),
            ('REMIND-MAgPIE 2.1-4.3', 'DeepElec_SSP2_HighRE_Budg900', ...),
            ('REMIND-MAgPIE 2.1-4.3', 'DeepElec_SSP2_HighRE_Budg900', ...),
            ('REMIND-MAgPIE 2.1-4.3', 'DeepElec_SSP2_HighRE_Budg900', ...),
            ('REMIND-MAgPIE 2.1-4.3', 'DeepElec_SSP2_HighRE_Budg900', ...),
            ('REMIND-MAgPIE 2.1-4.3', 'DeepElec_SSP2_HighRE_Budg900', ...),
            ('REMIND-MAgPIE 2.1-4.3', 'DeepElec_SSP2_HighRE_Budg900', ...),
            ('REMIND-MAgPIE 2.1-4.3', 'DeepElec_SSP2_HighRE_Budg900', ...),
            ('REMIND-MAgPIE 2.1-4.3', 'DeepElec_SSP2_HighRE_Budg900', ...),
            ('REMIND-MAgPIE 2.1-4.3', 'DeepElec_SSP2_HighRE_Budg900', ...),
            ('REMIND-MAgPIE 2.1-4.3', 'DeepElec_SSP2_HighRE_Budg900', ...)],
           names=['model', 'scenario', 'fuel'])

In [35]:
projectlevel(fossil_series.index, ["model", "scenario", "fuel"]).to_frame(index=False)

,model,scenario,fuel
0,REMIND-MAgPIE 2.1-4.3,DeepElec_SSP2_HighRE_Budg900,Coal
1,REMIND-MAgPIE 2.1-4.3,DeepElec_SSP2_HighRE_Budg900,Coal
2,REMIND-MAgPIE 2.1-4.3,DeepElec_SSP2_HighRE_Budg900,Coal
3,REMIND-MAgPIE 2.1-4.3,DeepElec_SSP2_HighRE_Budg900,Coal
4,REMIND-MAgPIE 2.1-4.3,DeepElec_SSP2_HighRE_Budg900,Gas
5,REMIND-MAgPIE 2.1-4.3,DeepElec_SSP2_HighRE_Budg900,Gas
6,REMIND-MAgPIE 2.1-4.3,DeepElec_SSP2_HighRE_Budg900,Gas
7,REMIND-MAgPIE 2.1-4.3,DeepElec_SSP2_HighRE_Budg900,Gas
8,REMIND-MAgPIE 2.1-4.3,DeepElec_SSP2_HighRE_Budg900,Nuclear
9,REMIND-MAgPIE 2.1-4.3,DeepElec_SSP2_HighRE_Budg900,Nuclear
